In [12]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [59]:
!pip install nltk


In [26]:
# Set up Kaggle API key
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Download dataset (replace 'dataset-owner/dataset-name' with actual dataset path)
api.dataset_download_files('nikhiljohnk/tweets-with-sarcasm-and-irony', path='/content', unzip=True)


Dataset URL: https://www.kaggle.com/datasets/nikhiljohnk/tweets-with-sarcasm-and-irony


In [15]:
import kagglehub

In [27]:
import os
print(os.listdir('/content'))


['.config', 'kaggle .json', 'test.csv', 'train.csv', '.ipynb_checkpoints', 'your_desired_path', 'sample_data']


In [28]:
path = kagglehub.dataset_download("nikhiljohnk/tweets-with-sarcasm-and-irony")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/nikhiljohnk/tweets-with-sarcasm-and-irony/versions/3


In [29]:
import pandas as pd

# Define the path to the CSV file
csv_file_path = '/root/.cache/kagglehub/datasets/nikhiljohnk/tweets-with-sarcasm-and-irony/versions/3/sarcasm_data.csv'




In [23]:
import os
print(os.listdir('/content'))


['.config', 'kaggle (2).json', '.ipynb_checkpoints', 'your_desired_path', 'sample_data']


In [31]:
import pandas as pd

# Load the train and test datasets
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

# Check the first few rows of the train dataset
print(train_df.head())


                                              tweets       class
0  Be aware  dirty step to get money  #staylight ...  figurative
1  #sarcasm for #people who don't understand #diy...  figurative
2  @IminworkJeremy @medsingle #DailyMail readers ...  figurative
3  @wilw Why do I get the feeling you like games?...  figurative
4  -@TeacherArthurG @rweingarten You probably jus...  figurative


In [32]:
import os

# Check if the directory exists
dataset_directory = '/root/.cache/kagglehub/datasets/nikhiljohnk/tweets-with-sarcasm-and-irony/versions/3'
print(os.path.exists(dataset_directory))  # This will print True if the directory exists

# Check if the specific file exists
csv_file_path = '/root/.cache/kagglehub/datasets/nikhiljohnk/tweets-with-sarcasm-and-irony/versions/3/sarcasm_data.csv'
print(os.path.exists(csv_file_path))  # This will print True if the file exists


True
False


In [33]:
# Check for missing values
print(train_df.isnull().sum())


tweets    0
class     0
dtype: int64


In [34]:
# Count the number of figurative and non-figurative tweets
print(train_df['class'].value_counts())


class
figurative    21238
irony         20894
sarcasm       20681
regular       18595
Name: count, dtype: int64


In [35]:
import re

In [36]:
def clean_text(text):
    # Remove links
    text = re.sub(r"http\S+", "", text)
    # Remove special characters and numbers
    text = re.sub(r"[^A-Za-z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    return text

In [37]:
train_df['cleaned_tweets'] = train_df['tweets'].apply(clean_text)

In [38]:
print(train_df[['tweets', 'cleaned_tweets']].head())

                                              tweets  \
0  Be aware  dirty step to get money  #staylight ...   
1  #sarcasm for #people who don't understand #diy...   
2  @IminworkJeremy @medsingle #DailyMail readers ...   
3  @wilw Why do I get the feeling you like games?...   
4  -@TeacherArthurG @rweingarten You probably jus...   

                                      cleaned_tweets  
0  be aware  dirty step to get money  staylight s...  
1  sarcasm for people who dont understand diy art...  
2  iminworkjeremy medsingle dailymail readers bei...  
3  wilw why do i get the feeling you like games s...  
4  teacherarthurg rweingarten you probably just m...  


In [42]:
import os

# List files in the current directory
print(os.listdir())


['.config', 'kaggle .json', 'test.csv', 'train.csv', '.ipynb_checkpoints', 'your_desired_path', 'sample_data']


In [72]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [73]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters, links, and usernames (if any)
    text = re.sub(r'(@\w+|https?://\S+|www\.\S+)', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

In [75]:
train_df['cleaned_tweet'] = train_df['tweets'].apply(preprocess_text)


In [76]:
X = train_df['cleaned_tweet']  # Features (text data)
y = train_df['class']  # Target (multiclass labels)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

In [77]:
vectorizer = TfidfVectorizer(max_features=5000)  # You can tune the number of features
X_tfidf = vectorizer.fit_transform(X)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [79]:
logreg = LogisticRegression(max_iter=1000, multi_class='ovr')  # One-vs-Rest approach
logreg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='ovr')

In [80]:
y_pred = logreg.predict(X_test)

In [81]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7207345534946566
Classification Report:
               precision    recall  f1-score   support

  figurative       0.25      0.03      0.05      4179
       irony       0.67      0.95      0.78      4276
     regular       0.96      1.00      0.98      3696
     sarcasm       0.66      0.94      0.77      4131

    accuracy                           0.72     16282
   macro avg       0.63      0.73      0.65     16282
weighted avg       0.62      0.72      0.64     16282



In [82]:
print("\nOriginal and Cleaned Tweets (sample):")
sample_tweets = train_df[['tweets', 'cleaned_tweet']].head(5)
print(sample_tweets)


Original and Cleaned Tweets (sample):
                                              tweets  \
0  Be aware  dirty step to get money  #staylight ...   
1  #sarcasm for #people who don't understand #diy...   
2  @IminworkJeremy @medsingle #DailyMail readers ...   
3  @wilw Why do I get the feeling you like games?...   
4  -@TeacherArthurG @rweingarten You probably jus...   

                                       cleaned_tweet  
0  aware dirty step get money staylight staywhite...  
1       sarcasm people dont understand diy artattack  
2  dailymail readers sensible always shocker sarc...  
3                     get feeling like games sarcasm  
4                     probably missed text sarcastic  


In [83]:
import joblib

In [84]:
joblib.dump(model, 'logistic_regression_model.pkl')

['logistic_regression_model.pkl']

In [85]:
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')  # Save the vectorizer

['tfidf_vectorizer.pkl']

In [86]:
model = joblib.load('logistic_regression_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

In [87]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [88]:
import joblib
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [89]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove non-alphabetic characters (special characters, numbers, etc.)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Return the cleaned text as a single string
    return " ".join(tokens)

In [96]:
model = joblib.load('logistic_regression_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Function to predict sarcasm
def predict_sarcasm(input_text, model, vectorizer):
    # Step 1: Preprocess the input text
    preprocessed_text = preprocess_text(input_text)

    # Step 2: Transform the preprocessed input text using the same TF-IDF vectorizer
    input_tfidf = vectorizer.transform([preprocessed_text])

    # Step 3: Make the prediction using the trained model
    prediction = model.predict(input_tfidf)

    # Return the predicted class
    return prediction[0]


In [99]:
import nltk
nltk.download('punkt')  # Ensure the 'punkt' tokenizer is downloaded
nltk.download('stopwords')  # Ensure stopwords are downloaded




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [100]:
# Preprocessing function
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters, links, and usernames (if any)
    text = re.sub(r'http\S+|www\S+|@[\w]+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

In [101]:
def predict_class(tweet, model, vectorizer):
    # Preprocess the input tweet
    cleaned_tweet = preprocess_text(tweet)

    # Vectorize the tweet using the trained TF-IDF vectorizer
    tweet_tfidf = vectorizer.transform([cleaned_tweet])

    # Predict the class using the trained model
    predicted_class = model.predict(tweet_tfidf)

    return predicted_class[0]

In [102]:
# Sample tweet (custom input)
custom_tweet = input("Enter a tweet for classification: ")

# Use the trained model and vectorizer to predict the class
predicted_class = predict_class(custom_tweet, logreg, vectorizer)

# Print the predicted class
print(f"The predicted class for the input tweet is: {predicted_class}")

Enter a tweet for classification: oh great another monday 
The predicted class for the input tweet is: sarcasm
